#### Engenharia de dados
**Nome**: MÁRCIO DE LIMA LOSS PUGNAL  
**Código Aluno**: 4052025000363  
**Curso**: Pós CIÊNCIA DE DADOS E ANALYTICS  
**Dataset**: [Board-Games] [fonte: https://www.kaggle.com/datasets/andrewmvd/board-games ]

#### Solução das questões analíticas

**01. Avaliação e Popularidade dos Jogos:**  
_Quais são os jogos (top 10) de tabuleiro mais bem avaliados e sua relação com a popularidade?_

In [0]:
-- 01. Top 10 jogos de tabuleiro bem avaliados e mais populares:
--CREATE VIEW tabuleiro.vw01_popularidade_avaliacao AS
SELECT 
    dj.nome_jogo          AS Nome_Jogo,
    dj.ano_publicacao     AS Ano_Publicacao,
    fa.media_avaliacao    AS Avaliacao_Media,
    fa.usuarios_avaliaram AS Total_Avaliacoes,
    fa.usuarios_possuem   AS Total_Proprietarios
FROM tabuleiro.fato_avaliacao fa
INNER JOIN tabuleiro.dim_jogo dj ON fa.id_jogo = dj.id_jogo
WHERE fa.usuarios_possuem > 0 AND 
      fa.media_avaliacao > 0
    ORDER BY fa.media_avaliacao DESC, fa.usuarios_possuem DESC
LIMIT 10;

Nome_Jogo,Ano_Publicacao,Avaliacao_Media,Total_Avaliacoes,Total_Proprietarios
Company of Heroes,2020,9.34,47,122
Wings of the Motherland,2019,9.31,79,211
Gladiatoris,2009,9.23,31,7
Primer: The Gamer's Source for Battles from the Age of Reason,2013,9.14,58,255
1985: Under an Iron Sky,2018,9.12,90,363
Global War 1939,2011,9.05,34,55
Arena: The Contest,2019,8.99,600,1094
Dungeon Universalis,2019,8.99,454,697
1985: Deadly Northern Lights,2020,8.99,36,155
Europa Universalis: The Price of Power,2021,8.98,74,103


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_- Os jogos mais bem avaliados não são necessariamente os mais populares._  
_- A maioria dos jogos com notas altas é relativamente recente (2018–2021)._  
_- Jogos com muitas avaliações tendem a ter notas ligeiramente menores, mas mais confiáveis estatisticamente._  
_- Jogos com poucas avaliações podem inflar notas, mas não representam consenso estatístico._  
_- Jogos com um volume maior de avaliações tendem a ter notas ligeiramente mais moderadas, porém estatisticamente mais confiáveis, refletindo um consenso mais amplo._

**02. Complexidade e Avaliação dos Jogos:**  
_Como o nível de complexidade dos jogos influencia a avaliação média dos jogos?_

In [0]:
-- 02. Nível de complexidade por média de avaliação dos jogos
--CREATE VIEW tabuleiro.vw02_complexidade_avaliacao AS
SELECT 
    CASE 
        WHEN fa.media_complexidade < 2 THEN 'Iniciante (0-2)'
        WHEN fa.media_complexidade < 3 THEN 'Intermediário (2-3)'
        WHEN fa.media_complexidade < 4 THEN 'Avançado (3-4)'
        ELSE 'Expert (4-5)'
    END AS Nivel_Complexidade,
    ROUND(AVG(fa.media_avaliacao), 2) AS Media_Avaliacao,
    COUNT(*) AS Total_Jogos
FROM tabuleiro.fato_avaliacao fa
WHERE fa.media_complexidade IS NOT NULL
GROUP BY Nivel_Complexidade
ORDER BY Media_Avaliacao ASC;

Nivel_Complexidade,Media_Avaliacao,Total_Jogos
Iniciante (0-2),6.12,3513
Intermediário (2-3),6.76,3946
Avançado (3-4),7.12,1909
Expert (4-5),7.54,341


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_- Jogos mais complexos (avançado e expert) tendem a receber avaliações mais altas. Há uma clara correlação positiva: quanto maior a complexidade, maior a média de avaliação._  
_- A maior parte dos jogos está concentrada nos níveis Iniciante e Intermediário (mais de 7.400 títulos)._  
_- Jogos simples são abundantes, mas não tão bem avaliados._  
_- Jogos complexos são menos numerosos, mas recebem notas mais altas._

**03. Mecânicas de Jogo: Avaliação e Popularidade**  
_Quais as mecânicas de jogos que estão associadas às maiores avaliações e popularidade?_

In [0]:
-- 03. Mecânicas mais bem avaliadas e sua popularidade.
-- CREATE VIEW tabuleiro.vw03_mecanicas_avaliadas_populares AS
SELECT 
    dm.mecanica_jogo AS Mecanica_Jogo,
    ROUND(AVG(fa.media_avaliacao), 2) AS Avaliacao_Media,
    SUM(fa.usuarios_possuem) AS Popularidade,
    COUNT(DISTINCT fa.id_jogo) AS Total_Jogos
FROM tabuleiro.fato_avaliacao fa
JOIN tabuleiro.dim_mecanicas dm 
    ON fa.id_jogo = dm.id_jogo
GROUP BY dm.mecanica_jogo
ORDER BY Avaliacao_Media DESC, Popularidade DESC
LIMIT 10;

Mecanica_Jogo,Avaliacao_Media,Popularidade,Total_Jogos
Legacy Game,7.87,280037,20
Automatic Resource Growth,7.82,228094,11
Ownership,7.78,311224,27
Delayed Purchase,7.77,258303,10
Turn Order: Pass Order,7.75,176496,12
Turn Order: Auction,7.74,146152,10
Turn Order: Claim Action,7.7,474439,28
Victory Points as a Resource,7.68,476511,36
Command Cards,7.68,93200,10
Market,7.66,322978,35


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_As mecânicas listadas têm sua avaliação próxima, sem grandes discrepâncias, o que indica homogeneidade na avaliação._  
_Não há mecânicas com notas muito baixas ou muito altas._  
_As mecânicas mais populares não são necessariamente as mais bem avaliadas._  
_Popularidade não garante avaliação máxima._

**04. Complexidade e Duração das Partidas**  
_Qual a correlação entre a complexidade dos jogos e o tempo médio de uma partida?_

In [0]:
-- 04. Complexidade média e tempo de jogo
-- CREATE VIEW tabuleiro.vw04_complexidade_tempo_jogo AS
SELECT 
    CASE 
        WHEN fa.media_complexidade < 2 THEN 'Baixa (0-2)'
        WHEN fa.media_complexidade < 3 THEN 'Média (2-3)'
        WHEN fa.media_complexidade < 4 THEN 'Alta (3-4)'
        ELSE 'Muito Alta (4-5)'
    END AS Faixa_Complexidade,
    ROUND(AVG(djogab.tempo_jogo), 2) AS Tempo_Medio_Jogo,
    ROUND(AVG(fa.media_avaliacao), 2) AS Avaliacao_Media,
    COUNT(DISTINCT fa.id_jogo) AS Total_Jogos
FROM tabuleiro.fato_avaliacao fa
JOIN tabuleiro.dim_jogabilidade djogab 
    ON fa.id_jogo = djogab.id_jogo
GROUP BY Faixa_Complexidade
ORDER BY Tempo_Medio_Jogo ASC;

Faixa_Complexidade,Tempo_Medio_Jogo,Avaliacao_Media,Total_Jogos
Baixa (0-2),35.2,6.12,3513
Média (2-3),101.05,6.76,3946
Alta (3-4),234.24,7.12,1909
Muito Alta (4-5),1073.74,7.54,341


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_Jogos com baixa e média complexidade são jogos rápidos, mas esses não alcançam notas tão altas._  
_Têm acesso a uma grande oferta de jogos rápidos e simples, mas esses não alcançam notas tão altas._  
_Jogos de muito alta complexidade são raros (apenas 341), mas recebem a melhor avaliação média._  
_Jogadores tendem a valorizar mais jogos com alta complexidade, pode ser por ser mais desafiador e profundo, mesmo que exijam mais tempo._

**05. Avaliação por Faixa Etária Recomendada**  
_Como a faixa etária recomendada pelo jogo influencia a avaliação dos jogos?_

In [0]:
-- 05. Comparação de avaliação por faixa etária recomendada
-- CREATE VIEW tabuleiro.vw05_avaliacao_faixa_etaria AS
SELECT 
    CASE 
        WHEN djg.idade_minima BETWEEN 0 AND 5  THEN 'Infantil (0-5)'
        WHEN djg.idade_minima BETWEEN 6 AND 12 THEN 'Pré-adolescente (6-12)'
        WHEN djg.idade_minima BETWEEN 13 AND 17 THEN 'Adolescente (13-17)'
        ELSE 'Adulto (18+)' 
    END AS Faixa_Etaria_Jogador,
    ROUND(AVG(fa.media_avaliacao), 2) AS Media_Avaliacao_Jogo,
    COUNT(DISTINCT fa.id_jogo) AS Total_Jogos
FROM tabuleiro.fato_avaliacao fa
JOIN tabuleiro.dim_jogabilidade djg 
    ON fa.id_jogo = djg.id_jogo
GROUP BY Faixa_Etaria_Jogador
ORDER BY Media_Avaliacao_Jogo DESC;


Faixa_Etaria_Jogador,Media_Avaliacao_Jogo,Total_Jogos
Adolescente (13-17),7.14,1545
Pré-adolescente (6-12),6.54,7064
Infantil (0-5),6.43,1065
Adulto (18+),6.29,35


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_Melhor avaliação entre adolescentes (13-17)._  
_Adolescentes são o público que mais valoriza os jogos disponíveis._  
_Maior oferta de jogos para pré-adolescentes(6–12)._  
_Mostra que há pouquíssimos jogos para adultos (18+) e avaliação não é tão alta, embora seja a menor nota, ela está relativamente próxima das demais faixas._

**06. Dominância e Qualidade por Categoria Temática**  
_Quais categorias (dominios) temáticas tem maior popularidade e mantêm alta avaliação?_

In [0]:
-- 06. Domínios mais populares e bem avaliados
-- CREATE VIEW tabuleiro.vw06_dominios_populares AS
SELECT 
    dd.dominio_jogo              AS Dominio_Jogo,
    SUM(fb.usuarios_possuem)     AS Total_Proprietarios,
    ROUND(AVG(fb.media_avaliacao), 2) AS Avaliacao_Media,
    COUNT(DISTINCT fb.id_jogo)   AS Total_Jogos
FROM tabuleiro.fato_avaliacao fb
JOIN tabuleiro.dim_dominios dd ON fb.id_jogo = dd.id_jogo
WHERE fb.media_avaliacao IS NOT NULL
GROUP BY dd.dominio_jogo
ORDER BY Avaliacao_Media DESC, Total_Proprietarios DESC;

Dominio_Jogo,Total_Proprietarios,Avaliacao_Media,Total_Jogos
Strategy Games,9773182,6.96,2182
Wargames,2330119,6.85,3287
Thematic Games,4727819,6.81,1142
Customizable Games,653285,6.52,252
Family Games,8726469,6.43,2072
Party Games,2399690,6.31,550
Abstract Games,1408494,6.27,940
Children's Games,564179,5.53,728


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_Strategy Games - tem alta popularidade (9,7 milhões de proprietários) e têm boa avaliação (6.96), equilibrando alcance e qualidade._  
_Family Games - também muito populares (8,7 milhões), mas com avaliação mediana (6.43), mostrando que volume não garante qualidade._  
_Wargames - maior número de títulos (3.287), avaliação sólida (6.85), mas público menos massivo._  
_Thematic Games - boa avaliação (6.81) e popularidade intermediária, valorizados por públicos específicos._  
_Customizable Games - poucos títulos (252), mas comunidades fiéis (650 mil proprietários)._  
_Party Games e Abstract Games - populares em contextos sociais ou conceituais, mas com notas abaixo de 6.5._  
_Children’s Games - menor avaliação (5.53) e baixa popularidade, indicando espaço para inovação no segmento infantil._  

**07. Duração Ideal vs. Avaliação do Jogador**  
_Qual a relação entre tempo de jogo e satisfação dos jogadores?_

In [0]:
-- 07. Tempo de jogo vs. Avaliação
-- CREATE VIEW tabuleiro.vw07_tempo_jogo_avaliacao AS
SELECT 
    CASE 
        WHEN j.tempo_jogo <= 30 THEN 'Curto (≤30 min)'
        WHEN j.tempo_jogo BETWEEN 31 AND 60 THEN 'Médio (31-60 min)'
        WHEN j.tempo_jogo BETWEEN 61 AND 120 THEN 'Alto (61-120 min)'
        ELSE 'Longo (>120 min)'
    END AS faixa_tempo,
    ROUND(AVG(f.media_avaliacao),2) AS media_avaliacao,
    COUNT(DISTINCT f.id_jogo) AS total_jogos
FROM tabuleiro.fato_avaliacao f
JOIN tabuleiro.dim_jogabilidade j ON f.id_jogo = j.id_jogo
GROUP BY faixa_tempo
ORDER BY media_avaliacao DESC;

faixa_tempo,media_avaliacao,total_jogos
Longo (>120 min),7.06,1997
Alto (61-120 min),6.84,2117
Médio (31-60 min),6.63,2499
Curto (≤30 min),6.2,3096


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_Jogos longos (>120 min) tendem a ter notas mais altas._  
_Jogos curtos (≤30 min) são abundantes, mas menos valorizados._  
_Satisfação cresce com maior investimento de tempo, sugerindo que jogadores associam duração a profundidade._

**08. Faixas de jogadores: popularidade, avaliação e tempo médio.**  
_Qual é o perfil dos jogos (quantidade, avaliação e tempo) por faixa de jogadores?_

In [0]:
-- 08. Perfil dos jogos por faixa de jogadores.
-- CREATE VIEW tabuleiro.vw08_perfil_faixa_jogadores AS
SELECT
    CASE
        WHEN jog.max_jogadores <= 2 THEN '1-2 jogadores'
        WHEN jog.max_jogadores <= 4 THEN '3-4 jogadores'
        WHEN jog.max_jogadores <= 6 THEN '5-6 jogadores'
        ELSE '7+ jogadores'
    END AS faixa_jogadores,
    COUNT(fa.id_jogo) AS total_jogos,
    ROUND(AVG(fa.media_avaliacao), 2) AS media_avaliacao,
    ROUND(AVG(jog.tempo_jogo), 2) AS media_tempo_jogo
FROM
    tabuleiro.fato_avaliacao fa JOIN
    tabuleiro.dim_jogabilidade jog ON fa.id_jogo = jog.id_jogo
GROUP BY
    faixa_jogadores
ORDER BY
    total_jogos DESC;

faixa_jogadores,total_jogos,media_avaliacao,media_tempo_jogo
1-2 jogadores,3148,6.74,170.39
3-4 jogadores,3026,6.62,105.37
5-6 jogadores,2656,6.54,115.6
7+ jogadores,879,6.5,197.31


Databricks visualization. Run in Databricks to view.

📌 **Comentário**:  
_Jogos para poucos jogadores são os mais bem avaliados._  
_Verificado que a concentram em faixas de até 4 jogadores, que somam mais de 6.000 títulos._  
_Grupos grandes (5+) têm menos opções, partidas longas e avaliações mais baixas._  

📋 **Conclusão Geral:**  
_Popularidade e Qualidade: jogos mais jogados não são necessariamente os mais bem avaliados._  
_Complexidade e duração: são fatores que elevam a nota média._  
_Faixa etária e número de jogadores revelam lacunas: adultos e grupos grandes têm menos opções e menor satisfação._  
_Categorias e mecânicas mostram fidelidade de nichos, mas não garantem notas altas._  